# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
# загрузка библиотек
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statistics import mean
from numpy.random import RandomState
from scipy import stats as st

In [2]:
# чтение файла с данными и сохранение в переменную df
try:
    df0 = pd.read_csv('C:/Users/79853/Documents/datascience/yandex/Спринт9/geo_data_0.csv')
    df1 = pd.read_csv('C:/Users/79853/Documents/datascience/yandex/Спринт9/geo_data_1.csv')
    df2 = pd.read_csv('C:/Users/79853/Documents/datascience/yandex/Спринт9/geo_data_2.csv')
except:
    df0 = pd.read_csv('/datasets/geo_data_0.csv') # загрузка онлайн
    df1 = pd.read_csv('/datasets/geo_data_1.csv') # загрузка онлайн
    df2 = pd.read_csv('/datasets/geo_data_2.csv') # загрузка онлайн

In [3]:
data = [df0, df1, df2]
for df in data:
    print(df.info())
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column

In [4]:
for df in data:
    print(df.describe().T)
    print()

            count       mean        std        min        25%        50%  \
f0       100000.0   0.500419   0.871832  -1.408605  -0.072580   0.502360   
f1       100000.0   0.250143   0.504433  -0.848218  -0.200881   0.250252   
f2       100000.0   2.502647   3.248248 -12.088328   0.287748   2.515969   
product  100000.0  92.500000  44.288691   0.000000  56.497507  91.849972   

                75%         max  
f0         1.073581    2.362331  
f1         0.700646    1.343769  
f2         4.715088   16.003790  
product  128.564089  185.364347  

            count       mean        std        min        25%        50%  \
f0       100000.0   1.141296   8.965932 -31.609576  -6.298551   1.153055   
f1       100000.0  -4.796579   5.119872 -26.358598  -8.267985  -4.813172   
f2       100000.0   2.494541   1.703572  -0.018144   1.000021   2.011479   
product  100000.0  68.825000  45.944423   0.000000  26.953261  57.085625   

                75%         max  
f0         8.621015   29.421755  

In [5]:
for df in data:
    print(df.head())
    print()

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746



В файлах находятся данные геологоразведки трёх регионов:
- **id** — уникальный идентификатор скважины;
- **f0, f1, f2** — три признака точек (неважно, что они означают, но сами признаки значимы);
- **product** — объём запасов в скважине (тыс. баррелей).

In [6]:
# подсчёт явных дубликатов
for df in data:
    print(df.duplicated().sum())

0
0
0


### Вывод

Все три таблицы имеют одинаковую структуру и одинаковое количество данных (по 100 000).

Признаки `f0`, `f1`, `f2`, `product` - количественные. Признак `id` - категориальный.

Признак `id` (уникальный идентификатор скважины) не будет использоваться в исследование, поле можно удалить.

Пропуски и явные дубликаты в данных отсутствуют.

## Обучение и проверка модели

В качестве модели будет использована модель линейной регрессии (LinearRegression).

Данные будут разбиты на обучающую и валидационную выборки в соотношении 75:25.

Признаки - `f0`, `f1`, `f2`.

Целевой признак - `product`.

In [7]:
# функция для разделения датасета на обучающую и валидационную выборки, на признаки и целевой признак
def split_data(data):
    train_df, valid_df = train_test_split(data, test_size=0.25, random_state=123) # обучающая и валидационная - 0,75 и 0,25
    
    # разделение выборок на признаки (features) и целевой признак (target)
    features_train = train_df.drop(['product', 'id'], axis=1)
    target_train = train_df['product']

    features_valid = valid_df.drop(['product', 'id'], axis=1)
    target_valid = valid_df['product']
    
    return features_train, target_train, features_valid, target_valid

In [8]:
# функция для создания и обучения модели линейной регрессии
# выводит среднее значение предсказаний и среднеквадратическую ошибку
def mean_stats(data):
    features_train, target_train, features_valid, target_valid = split_data(data)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    mean_prediction = mean(predictions)
    rmse = mean_squared_error(target_valid, predictions) ** 0.5
    print('Средний запас предсказанного сырья:', round(mean_prediction, 3), '(тыс. барелей)')
    print('Среднеквадратическая ошибка RMSE:', round(rmse, 2))

In [9]:
# Первый регион
mean_stats(df0)

Средний запас предсказанного сырья: 92.549 (тыс. барелей)
Среднеквадратическая ошибка RMSE: 37.65


In [10]:
# Второй регион
mean_stats(df1)

Средний запас предсказанного сырья: 69.28 (тыс. барелей)
Среднеквадратическая ошибка RMSE: 0.9


In [11]:
# Третий регион
mean_stats(df2)

Средний запас предсказанного сырья: 95.099 (тыс. барелей)
Среднеквадратическая ошибка RMSE: 40.13


### Вывод

Самый высокий показатель среднего запаса пресказанного сырья (95,1) у Третьего региона, однако здесь довольно высокое значение среднеквадратической ошибки RMSE равное 40,1.

У Первого региона показатели немного ниже, чем у Третьего района, но по соотношению здесь ситуация аналогичная Третьему региону - довольно высокая RMSE.

У Второго региона показатель среднего запаса пресказанного сырья (69,3) самый низкий, однако значение среднеквадратической ошибки RMSE всего 0,9, т.е. разброс значений гораздо меньше, чем у Первого и Третьего регионов.

Исходя из полученных значений, предварительно можно сказать, что **Второй регион является самым предсказуемым для разработки**, т.к. предсказанное значение запаса сырья во всех его скважинах примерно одинаковое. 

## Подготовка к расчёту прибыли

Введем все необходимые переменные для дальнейших рассчетов.

In [12]:
RESEARCH_POINTS = 500    # число точек, которое исследуют при разведке региона
DEVELOPMEN_POINTS = 200  # число лучших точек для разработки
BUDGET = 10**10          # бюджет на разработку скважин в регионе в рублях
PRICE_PER_ITEM = 450000  # доход с каждой единицы продукта в рублях
RISK = 0.025             # максимальная вероятность убытков 

In [13]:
# минимальный объём запасов в скважине (тыс. баррелей) для безубыточной работы
min_profit_product = BUDGET / DEVELOPMEN_POINTS / PRICE_PER_ITEM
min_profit_product

111.11111111111111

### Вывод

Достаточный объём сырья для безубыточной разработки новой скважины составляет 111 тыс. барелей. Рассчитанный объём сырья превышает средние значения запасов скважин во всех исследуемых регионах.

## Расчёт прибыли и рисков 

Составим функцию для расчета валовой прибыли. Формула валовой прибыли выглядит следующим образом:

**Валовая прибыль = Выручка - Себестоимость**

В условиях задачи:
- **Выручка** - это доход от продажи нефти с лучших скважин региона, которые будут разработаны.
- **Себестоимость** - это бюджет на разработку скважин.

In [14]:
# функция для расчета максимальной валовой прибыли
def gross_profit(data):
    features_train, target_train, features_valid, target_valid = split_data(data)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    target_valid = pd.Series(target_valid).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    pred_sorted = predictions.sort_values(ascending=False)
    selected = target_valid[pred_sorted.index][:DEVELOPMEN_POINTS]
    return PRICE_PER_ITEM * selected.sum() - BUDGET

In [15]:
print("Валовая прибыль с 200 лучших скважин в Первом регионе:",
      '{0:,}'.format(int(gross_profit(df0))).replace(',', ' '), 'руб.')
print("Валовая прибыль с 200 лучших скважин во Втором регионе:",
      '{0:,}'.format(int(gross_profit(df1))).replace(',', ' '), 'руб.')
print("Валовая прибыль с 200 лучших скважин в Третьем регионе:",
      '{0:,}'.format(int(gross_profit(df2))).replace(',', ' '), 'руб.')

Валовая прибыль с 200 лучших скважин в Первом регионе: 3 534 670 917 руб.
Валовая прибыль с 200 лучших скважин во Втором регионе: 2 415 086 696 руб.
Валовая прибыль с 200 лучших скважин в Третьем регионе: 2 370 343 863 руб.


Расчет показал, что наибольшую валовую прибыль в размере 3,53 млрд. рублей можно получить в Первом регионе. 

Составим функцию для расчета рисков и валовой прибыли для регионов. Чтобы найти распределение прибыли применим технику Bootstrap с 1000 выборок. Функция будет выводить:
- Среднюю валовую прибыль в рублях
- 95%-й доверительный интевал для валовой прибыли
- Вероятность убытков в %

In [16]:
# функция для расчета рисков и прибыли
def risks(data):
    features_train, target_train, features_valid, target_valid = split_data(data)
    target_valid = target_valid.reset_index(drop=True)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = pd.Series(model.predict(features_valid))
   
    state = np.random.RandomState(12345)
    profits = []     # выручка
    
    for i in range(1000):
        target_subsample = target_valid.sample(n=RESEARCH_POINTS, replace=True, random_state=state)  # 500 случайных ответов
        probs_subsample = predictions[target_subsample.index]     # 500 предсказаний соответсвующие 500 случайных ответов
        
        pred_sorted = probs_subsample.sort_values(ascending=False)  #сортировка по убыванию выбранных 500 предсказаний
        selected = target_subsample[pred_sorted.index][:DEVELOPMEN_POINTS] #выборка 200 ответов по отсортированным предсказаниям
        
        result = PRICE_PER_ITEM * selected.sum() - BUDGET
        profits.append(result)
                  
    profits = pd.Series(profits)
    damages_probability = (profits < 0).mean()     # вероятность убытков
    
    # 95%-й доверительный интервал
    confidence_interval = (profits.quantile(0.025), profits.quantile(0.975))
    
    print('Средняя валовая прибыль:', '{0:,}'.format(int(profits.mean())).replace(',', ' '), 'руб.')
    print('95%-й доверительный интевал для валовой прибыли (руб.):', confidence_interval)
    print("Вероятность убытков: {:.1%}".format(damages_probability)) 

    if damages_probability < RISK:
        print('Вероятность убытков ниже установленного порога')
    elif damages_probability == RISK:
        print('Вероятность убытков равна установленному порогу')
    else:
        print('Вероятность убытков выше установленного порога')

In [17]:
# Первый регион
risks(df0)

Средняя валовая прибыль: 505 264 703 руб.
95%-й доверительный интевал для валовой прибыли (руб.): (-1086050.0993474456, 1062107249.360262)
Вероятность убытков: 2.6%
Вероятность убытков выше установленного порога


In [18]:
# Второй регион
risks(df1)

Средняя валовая прибыль: 541 346 019 руб.
95%-й доверительный интевал для валовой прибыли (руб.): (112268493.96322638, 1003682292.7565937)
Вероятность убытков: 0.6%
Вероятность убытков ниже установленного порога


In [19]:
# Третий регион
risks(df2)

Средняя валовая прибыль: 377 594 883 руб.
95%-й доверительный интевал для валовой прибыли (руб.): (-196111540.74364287, 943136714.1862346)
Вероятность убытков: 9.4%
Вероятность убытков выше установленного порога


### Вывод

По всем рассчитанным параметрам лидирует **Второй регион**. Здесь самое высокое среди регионов значение средней валовой прибыли и вероятность убытков ниже установленного порога в 2,5%.

Можно сделать вывод, что **самым выгодным регионом для разработки скважин является Второй регион**.

## Общий вывод

В ходе проекта были исследованы данные о пробах нефти в трех регионах с целью анализа возможной прибыли и рисков. 

Для построения модели, которая предсказывает объём запасов в скважинах, был использован алгоритм линейной регрессии (**LinearRegression**). Данные были разбиты на обучающую и валидационную выборки в соотношении 75:25.

В п.2 для каждого региона были рассчитаны **средний запас предсказанного сырья** и **среднеквадратическая ошибка RMSE**. По полученным результатам было сделано предположение, что Второй регион является самым предсказуемым для разработки, т.к. предсказанное значение запаса сырья во всех его скважинах примерно одинаковое, среднеквадратическая ошибка RMSE меньше единицы.

В п.3 были подготовлены необходимые переменные с ключевыми значениями для расчётов. Также был рассчитан **минимальный объем запасов в скважине**, необходимый для безубыточной работы. Оказалось, что рассчитанный минимальный объем превышает средние значения запасов скважин во всех исследуемых регионах.

В п.4 была рассчитана **максимальная валовая прибыль с 200 лучших скважин** для каждого региона. Расчет показал, что наибольшую валовую прибыль размером 3,53 млрд. рублей можно получить в Первом регионе. Далее была применена техника **Bootstrap** для поиска распределения прибыли. Т.о. была рассчитана **средняя валовая прибыль**, **95%-й доверительный интевал для валовой прибыли** и **вероятность убытков**.

Исходя из результатов исследования, был сделан вывод, что **самым выгодным регионом для разработки скважин является Второй регион** с показателем средней валовой прибыли 541 346 019 руб. и вероятностью убытков 0.6%.